d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Relational Entities on Databricks

## Learning Objectives
**In this lab, you will create and explore interactions between various relational entities, including:**

- Databases
- Tables (managed and unmanaged)
- Views (views, temp views, and global temp views)

**Resources**
* [Databases and Tables - Databricks Docs](https://docs.databricks.com/user-guide/tables.html)
* [Managed and Unmanaged Tables](https://docs.databricks.com/user-guide/tables.html#managed-and-unmanaged-tables)
* [Creating a Table with the UI](https://docs.databricks.com/user-guide/tables.html#create-a-table-using-the-ui)
* [Create a Local Table](https://docs.databricks.com/user-guide/tables.html#create-a-local-table)
* [Saving to Persistent Tables](https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html#saving-to-persistent-tables)

-sandbox
### Getting Started

Run the following cell to configure our "classroom."

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> This script includes logic to declare a user-specific database. We won't be using this database for this lesson.

In [0]:
%run ./Includes/Classroom-Setup

Initialized classroom variables & functions...

Mounted datasets to /mnt/training from s3a://databricks-corp-training/common

Created user-specific database

Using the database juancgomdotpad_gmail_com_db .

All done!

## Overview of the Data

The data include multiple entries from a selection of weather stations, including average temperatures recorded in either Fahrenheit or Celcius. The schema for the table:

|ColumnName  | DataType| Description|
|------------|---------|------------|
|NAME        |string   | Station name |
|STATION     |string   | Unique ID |
|LATITUDE    |float    | Latitude |
|LONGITUDE   |float    | Longitude |
|ELEVATION   |float    | Elevation |
|DATE        |date     | YYYY-MM-DD |
|UNIT        |string   | Temperature units |
|TAVG        |float    | Average temperature |

The path to the data is provided below.

In [0]:
source = "dbfs:/mnt/training/weather/StationData/stationData.parquet"

Spark SQL allows queries directly against Parquet directories.

In [0]:
display(spark.sql(f"""
  SELECT * 
  FROM parquet.`{source}`
"""))

NAME,STATION,LATITUDE,LONGITUDE,ELEVATION,DATE,UNIT,TAVG
"HAYWARD AIR TERMINAL, CA US",USW00093228,37.6542,-122.115,13.1,2018-05-27,F,61.0
"BIG ROCK CALIFORNIA, CA US",USR0000CBIR,38.0394,-122.57,457.2,2018-01-05,C,11.7
"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",USW00023234,37.6197,-122.3647,2.4,2018-02-24,C,8.3
"LAS TRAMPAS CALIFORNIA, CA US",USR0000CTRA,37.8339,-122.0669,536.4,2018-03-26,C,9.4
"HOUSTON INTERCONTINENTAL AIRPORT, TX US",USW00012960,29.98,-95.36,29.0,2018-05-25,F,80.0
"BIG ROCK CALIFORNIA, CA US",USR0000CBIR,38.0394,-122.57,457.2,2018-05-16,C,11.1
"BLACK DIAMOND CALIFORNIA, CA US",USR0000CBKD,37.95,-121.8844,487.7,2018-05-25,C,10.6
"LAS TRAMPAS CALIFORNIA, CA US",USR0000CTRA,37.8339,-122.0669,536.4,2018-05-21,C,11.7
"WOODACRE CALIFORNIA, CA US",USR0000CWOO,37.9906,-122.6447,426.7,2018-05-26,F,53.0
"BRIONES CALIFORNIA, CA US",USR0000CBRI,37.9442,-122.1178,442.0,2018-04-08,F,53.0


## Configure a `dbname` Variable
This ensures we won't conflict with other databases and will make it easier to write checks with Python later in the lesson.

Suggested format: `<your_initials>_<some_integer>`, e.g., `foo_123`

In [0]:
# TODO
dbname = "yanse"

## Create a Database

Create a database in the default location using the `dbname` from the last cell.

In [0]:
# %sql
# -- TODO

spark.sql(f'CREATE DATABASE IF NOT EXISTS {dbname}')

Out[9]: DataFrame[]

## Change to Your New Database

`USE` your newly created database.

In [0]:
spark.sql(f'USE {dbname}')


Out[10]: DataFrame[]

## Create a Managed Table
Use a CTAS statement to create a managed table named `weather_managed`.

In [0]:
%sql
-- TODO

CREATE OR REPLACE TABLE weather_managed AS
SELECT * 
FROM parquet.`dbfs:/mnt/training/weather/StationData/stationData.parquet`

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.analysis.TableAlreadyExistsException: Table yanse.weather_managed already exists
	at org.apache.spark.sql.execution.datasources.v2.AtomicCreateTableAsSelectExec.run(WriteToDataSourceV2Exec.scala:121)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:41)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:41)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:47)
	at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:233)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3802)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:126)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:267)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:852)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:217)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:3800)
	at org.apache.spark.sql.Dataset. (Dataset.scala:233)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:103)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:852)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:100)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:687)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:852)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:682)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:694)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:144)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$13(DriverLocal.scala:544)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:53)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:279)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:271)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:53)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:521)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:689)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:681)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:522)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:634)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:427)
	at com.databricks.backend.d

## Create an Unmanaged Table

Unmanaged tables can be registered directly to existing directories of files. Create a table called `weather_unmanaged` using the provided location.

In [0]:
%sql
-- TODO

CREATE TABLE weather_unmanaged
USING parquet
LOCATION "dbfs:/mnt/training/weather/StationData/stationData.parquet"

## Examine Table Details
Use the SQL command `DESCRIBE EXTENDED table_name` to examine the two weather tables.

In [0]:
%sql

DESCRIBE EXTENDED weather_managed

col_name,data_type,comment
NAME,string,
STATION,string,
LATITUDE,float,
LONGITUDE,float,
ELEVATION,float,
DATE,date,
UNIT,string,
TAVG,float,
,,
# Partitioning,,


In [0]:
%sql

DESCRIBE EXTENDED weather_unmanaged

col_name,data_type,comment
NAME,string,null
STATION,string,null
LATITUDE,float,null
LONGITUDE,float,null
ELEVATION,float,null
DATE,date,null
UNIT,string,null
TAVG,float,null
,,
# Detailed Table Information,,


Run the following helper code to extract and compare the table locations.

In [0]:
def getTableLocation(tableName):
  return spark.sql(f"DESCRIBE EXTENDED {tableName}").select("data_type").filter("col_name = 'Location'").collect()[0][0]

In [0]:
managedTablePath = getTableLocation("weather_managed")
unmanagedTablePath = getTableLocation("weather_unmanaged")

print(f"""The weather_managed table is saved at: 

    {managedTablePath}

The weather_unmanaged table is saved at:

    {unmanagedTablePath}""")

The weather_managed table is saved at: 

 dbfs:/user/hive/warehouse/yanse.db/weather_managed

The weather_unmanaged table is saved at:

 dbfs:/mnt/training/weather/StationData/stationData.parquet

List the contents of these directories to confirm that data exists in both locations.

In [0]:
dbutils.fs.ls(managedTablePath)

Out[13]: [FileInfo(path='dbfs:/user/hive/warehouse/yanse.db/weather_managed/_delta_log/', name='_delta_log/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/yanse.db/weather_managed/part-00000-29c9e5a7-b90b-41f9-93d3-1483d2b16045-c000.snappy.parquet', name='part-00000-29c9e5a7-b90b-41f9-93d3-1483d2b16045-c000.snappy.parquet', size=7344),
 FileInfo(path='dbfs:/user/hive/warehouse/yanse.db/weather_managed/part-00001-c75abb48-9444-4011-badf-4423ba0011cb-c000.snappy.parquet', name='part-00001-c75abb48-9444-4011-badf-4423ba0011cb-c000.snappy.parquet', size=7294),
 FileInfo(path='dbfs:/user/hive/warehouse/yanse.db/weather_managed/part-00002-ff803666-6adb-4357-a35d-9bb9915f0ed7-c000.snappy.parquet', name='part-00002-ff803666-6adb-4357-a35d-9bb9915f0ed7-c000.snappy.parquet', size=7280),
 FileInfo(path='dbfs:/user/hive/warehouse/yanse.db/weather_managed/part-00003-957d6a8c-c296-44b2-b241-1688824967b9-c000.snappy.parquet', name='part-00003-957d6a8c-c296-44b2-b241-1688824967b9-c000.snappy.parquet', size=7285)]

In [0]:
dbutils.fs.ls(unmanagedTablePath)

Out[14]: [FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/_committed_6522815070831625438', name='_committed_6522815070831625438', size=420),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/_started_6522815070831625438', name='_started_6522815070831625438', size=0),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/part-00000-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3504-c000.snappy.parquet', name='part-00000-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3504-c000.snappy.parquet', size=6978),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/part-00001-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3505-c000.snappy.parquet', name='part-00001-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3505-c000.snappy.parquet', size=7035),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/part-00002-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3506-c000.snappy.parquet', name='part-00002-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3506-c000.snappy.parquet', size=6990),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/part-00003-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3507-c000.snappy.parquet', name='part-00003-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3507-c000.snappy.parquet', size=6975)]

-sandbox
### Check Directory Contents after Dropping Database and All Tables
The `CASCADE` keyword will accomplish this.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> You will encounter an error when listing your `managedTablePath`

In [0]:


spark.sql(f'DROP DATABASE {dbname} CASCADE')

Out[15]: DataFrame[]

In [0]:
dbutils.fs.ls(managedTablePath)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3964987377280387> in <module> 
 ----> 1 dbutils . fs . ls ( managedTablePath ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 328 exc . __context__ = None 
 329 exc . __cause__ = None 
 --> 330 raise exc
 331 return f_with_exception_handling
 332 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: /user/hive/warehouse/yanse.db/weather_managed
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:120)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:177)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:150)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCore.scala:154)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:91)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:153)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
dbutils.fs.ls(unmanagedTablePath)

Out[17]: [FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/_committed_6522815070831625438', name='_committed_6522815070831625438', size=420),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/_started_6522815070831625438', name='_started_6522815070831625438', size=0),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/part-00000-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3504-c000.snappy.parquet', name='part-00000-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3504-c000.snappy.parquet', size=6978),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/part-00001-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3505-c000.snappy.parquet', name='part-00001-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3505-c000.snappy.parquet', size=7035),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/part-00002-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3506-c000.snappy.parquet', name='part-00002-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3506-c000.snappy.parquet', size=6990),
 FileInfo(path='dbfs:/mnt/training/weather/StationData/stationData.parquet/part-00003-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3507-c000.snappy.parquet', name='part-00003-tid-6522815070831625438-4358ade9-a419-4a61-a2ea-9f64c9c3c0fc-3507-c000.snappy.parquet', size=6975)]

**This highlights the main differences between managed and unmanaged tables.** By default, the files associated with managed tables will be stored to this location on the root DBFS storage linked to the workspace, and will be deleted when a table is dropped.

Files for unmanaged tables will be persisted in the location provided at table creation, preventing users from inadvertently deleting underlying files. **Unmanaged tables can easily be migrated to other databases or renamed, but these operations with managed tables will require rewriting ALL underlying files.**

## Create a Database with a Specified Path

Assuming you dropped your database in the last step, you can use the same `dbname`.

In [0]:
spark.sql(f"CREATE DATABASE {dbname} LOCATION '{userhome}/{dbname}'")

Out[18]: DataFrame[]

Recreate your `weather_managed` table in this new database and print out the location of this table.

In [0]:
%sql
-- TODO
CREATE TABLE weather_managed AS
SELECT *
FROM parquet.`dbfs:/mnt/training/weather/StationData/stationData.parquet`

num_affected_rows,num_inserted_rows


In [0]:
getTableLocation("weather_managed")

Out[21]: 'dbfs:/user/juancgomdotpad@gmail.com/yanse/weather_managed'

While here we're using the `userhome` directory created on the DBFS root, _any_ object store can be used as the database directory. **Defining database directories for groups of users can greatly reduce the chances of accidental data exfiltration**.

## Views and their Scoping

Using the provided `AS` clause, register:
- a view named `celcius`
- a temporary view named `celcius_temp`
- a global temp view named `celcius_global`

In [0]:
%sql
-- TODO

CREATE OR REPLACE VIEW celcius
AS (SELECT *
  FROM weather_managed
  WHERE UNIT = "C")

In [0]:
%sql
-- TODO

CREATE OR REPLACE TEMP VIEW celcius_temp
AS (SELECT *
  FROM weather_managed
  WHERE UNIT = "C")

In [0]:
%sql
-- TODO

CREATE OR REPLACE GLOBAL TEMP VIEW celcius_global
AS (SELECT *
  FROM weather_managed
  WHERE UNIT = "C")

Views will be displayed alongside tables when listing from the catalog.

In [0]:
spark.catalog.listTables()

Out[22]: [Table(name='celcius', database='yanse', description=None, tableType='VIEW', isTemporary=False),
 Table(name='weather_managed', database='yanse', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='celcius_temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

Note the following:
- The view is associated with the current database. This view will be available to any user that can access this database and will persist between sessions.
- The temp view is not associated with any database. The temp view is ephemeral and is only accessible in the current SparkSession.
- The global temp view does not appear in our catalog. **Global temp views will always register to the `global_temp` database**. The `global_temp` database is ephemeral but tied to the lifetime of the cluster; however, it is only accessible by notebooks attached to the same cluster on which it was created.

In [0]:
%sql
SELECT * FROM global_temp.celcius_global

NAME,STATION,LATITUDE,LONGITUDE,ELEVATION,DATE,UNIT,TAVG
"BIG ROCK CALIFORNIA, CA US",USR0000CBIR,38.0394,-122.57,457.2,2018-01-05,C,11.7
"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",USW00023234,37.6197,-122.3647,2.4,2018-02-24,C,8.3
"LAS TRAMPAS CALIFORNIA, CA US",USR0000CTRA,37.8339,-122.0669,536.4,2018-03-26,C,9.4
"BIG ROCK CALIFORNIA, CA US",USR0000CBIR,38.0394,-122.57,457.2,2018-05-16,C,11.1
"BLACK DIAMOND CALIFORNIA, CA US",USR0000CBKD,37.95,-121.8844,487.7,2018-05-25,C,10.6
"LAS TRAMPAS CALIFORNIA, CA US",USR0000CTRA,37.8339,-122.0669,536.4,2018-05-21,C,11.7
"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",USW00023234,37.6197,-122.3647,2.4,2018-03-26,C,11.7
"LAS TRAMPAS CALIFORNIA, CA US",USR0000CTRA,37.8339,-122.0669,536.4,2018-01-30,C,15.0
"BLACK DIAMOND CALIFORNIA, CA US",USR0000CBKD,37.95,-121.8844,487.7,2018-04-30,C,10.6
"HOUSTON WILLIAM P HOBBY AIRPORT, TX US",USW00012918,29.63806,-95.28194,13.4,2018-04-03,C,23.9


While no job was triggered when defining these view, a job is triggered _each time_ a query is executed against the view.

## Clean Up
Drop the database and all tables to clean up your workspace.

In [0]:
#python
spark.sql(f"DROP DATABASE {dbname} CASCADE")

Out[23]: DataFrame[]

## Synopsis

In this lab we:
- Created and deleted databases
- Explored behavior of managed and unmanaged tables
- Learned about the scoping of views

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>